In [1]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 8.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=55c79742fb397ad4c1f3c1375894a4453452b8ee867a822086a3973029fecb63
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 10.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2")
print(model)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [6]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from typing import Any, Dict, List, Optional, Union
import torch

@dataclass
class DataCollatorForCausalLM:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        input_ids = [feature['input_ids'] for feature in features]
        labels = [feature['labels'] for feature in features]

        batch_input = self.tokenizer.pad(
            {'input_ids': input_ids},
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )

        batch_labels = self.tokenizer.pad(
            {'input_ids': labels},
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )

        batch_labels['input_ids'][batch_labels['input_ids'] == self.tokenizer.pad_token_id] = -100

        batch = {
            'input_ids': batch_input['input_ids'],
            'attention_mask': batch_input['attention_mask'],
            'labels': batch_labels['input_ids'],
        }

        return batch


In [11]:
from datasets import load_dataset
import random
import numpy as np
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

rouge = evaluate.load("rouge")

dataset = load_dataset("cnn_dailymail", "3.0.0")

num_train_samples = 27000
train_select = random.sample(range(len(dataset["train"])), k=num_train_samples)

num_validation_samples = 100
validation_select = random.sample(range(len(dataset["validation"])), k=num_validation_samples)

num_test_samples = 100
test_select = random.sample(range(len(dataset["test"])), k=num_test_samples)

dataset_train = dataset["train"].select(train_select)
dataset_validation = dataset["validation"].select(validation_select)
dataset_test = dataset["test"].select(test_select)

model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    prefix = "summarize: "
    inputs = [prefix + doc for doc in examples["article"]]
    targets = [ex for ex in examples["highlights"]]

    max_source_length = 512  
    max_target_length = 128  

    tokenized_inputs = tokenizer(
        inputs, max_length=max_source_length, truncation=True, padding=False
    )
    tokenized_targets = tokenizer(
        targets, max_length=max_target_length, truncation=True, padding=False
    )

    input_ids = []
    labels = []
    for i in range(len(tokenized_inputs["input_ids"])):
        input_ids_input = tokenized_inputs["input_ids"][i]
        input_ids_target = tokenized_targets["input_ids"][i]

        input_ids_combined = input_ids_input + input_ids_target
        input_ids_combined = input_ids_combined[:1024]  
        input_ids.append(input_ids_combined)

        labels_combined = [-100] * len(input_ids_input) + input_ids_target
        labels_combined = labels_combined[:1024]  
        labels.append(labels_combined)

    model_inputs = {
        "input_ids": input_ids,
        "labels": labels
    }

    return model_inputs

tokenized_train = dataset_train.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

tokenized_validation = dataset_validation.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["validation"].column_names,
)

tokenized_test = dataset_test.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["test"].column_names,
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["c_attn"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

data_collator = DataCollatorForCausalLM(tokenizer=tokenizer, padding='longest')

training_args = TrainingArguments(
    output_dir="gpt2_lora_test",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=2,  
    per_device_eval_batch_size=2,   
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    report_to="none",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    eval_accumulation_steps=2
)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

trainer.train()


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


trainable params: 589,824 || all params: 125,029,632 || trainable%: 0.4717


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.019300,1.973467,0.191700,0.088500,0.138100,0.180000,640.000000
2,2.294900,1.947959,0.193100,0.089700,0.138900,0.180600,640.000000
3,2.001500,1.934737,0.195000,0.090800,0.140800,0.182900,640.000000
4,2.242600,1.922721,0.195300,0.092400,0.142200,0.183700,640.000000
5,1.906900,1.918046,0.195000,0.091100,0.141200,0.182300,640.000000
6,1.894600,1.914614,0.192300,0.090700,0.139700,0.180600,640.000000
7,2.003600,1.909324,0.192900,0.090600,0.140400,0.180900,640.000000
8,1.984100,1.909751,0.193000,0.091900,0.140600,0.181700,640.000000
9,2.171300,1.908407,0.193300,0.091400,0.140600,0.182000,640.000000
10,2.020100,1.906913,0.192800,0.090800,0.140300,0.180900,640.000000


TrainOutput(global_step=135000, training_loss=2.1187491038004556, metrics={'train_runtime': 27440.0836, 'train_samples_per_second': 9.84, 'train_steps_per_second': 4.92, 'total_flos': 8.111897389023437e+16, 'train_loss': 2.1187491038004556, 'epoch': 10.0})

In [3]:
from datasets import load_dataset
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding, TrainingArguments
import numpy as np
import evaluate
from trl import SFTTrainer
import torch
import torch.nn as nn
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from typing import Any, Dict, List, Optional, Union
import torch

rouge = evaluate.load("rouge")

data_files = {"train": "train.csv", "test": "test.csv", "validation": "validation.csv"}
dataset = load_dataset("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail", data_files=data_files)
num_train_samples = 27000
train_select = random.sample(range(len(dataset["train"])), k=num_train_samples)

num_validation_samples = 100
validation_select = random.sample(range(len(dataset["validation"])), k=num_validation_samples)

num_test_samples = 100
test_select = random.sample(range(len(dataset["test"])), k=num_test_samples)

dataset_train = dataset["train"].select(train_select)
dataset_validation = dataset["validation"].select(validation_select)
dataset_test = dataset["test"].select(test_select)

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

num_soft_tokens = 20  
embedding_size = 768  

class GPT2WithSoftPrompt(nn.Module):
    def __init__(self, base_model, num_soft_tokens):
        super().__init__()
        self.base_model = base_model
        self.num_soft_tokens = num_soft_tokens
        self.soft_prompt_embeddings = nn.Parameter(torch.randn(num_soft_tokens, base_model.config.hidden_size))
        for param in self.base_model.parameters():
            param.requires_grad = False

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        input_embeds = self.base_model.transformer.wte(input_ids)
        batch_size = input_ids.size(0)
        soft_prompt_embeds = self.soft_prompt_embeddings.unsqueeze(0).expand(batch_size, -1, -1)
        inputs_embeds = torch.cat([soft_prompt_embeds, input_embeds], dim=1)
        if attention_mask is not None:
            soft_prompt_mask = torch.ones(batch_size, self.num_soft_tokens, dtype=attention_mask.dtype, device=attention_mask.device)
            attention_mask = torch.cat([soft_prompt_mask, attention_mask], dim=1)
        if labels is not None:
            soft_prompt_labels = torch.full((batch_size, self.num_soft_tokens), -100, dtype=labels.dtype, device=labels.device)
            labels = torch.cat([soft_prompt_labels, labels], dim=1)
        position_ids = torch.arange(inputs_embeds.size(1), dtype=torch.long, device=inputs_embeds.device)
        position_ids = position_ids.unsqueeze(0).expand(batch_size, -1)
        outputs = self.base_model(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            labels=labels,
            position_ids=position_ids,
        )
        return outputs

base_model = AutoModelForCausalLM.from_pretrained("gpt2")
model = GPT2WithSoftPrompt(base_model, num_soft_tokens=num_soft_tokens)


def preprocess_function(examples):
    inputs = examples["article"]
    model_inputs = tokenizer(inputs, max_length=400, truncation=True, padding='max_length')
    labels = tokenizer(text_target=examples["highlights"], max_length=400, truncation=True, padding='max_length')
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = dataset_train.map(preprocess_function, batched=True)
tokenized_validation = dataset_validation.map(preprocess_function, batched=True)
tokenized_test = dataset_test.map(preprocess_function, batched=True)

def compute_metrics(eval_pred):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids
    
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    
    predictions = predictions[:, num_soft_tokens:, :]
    labels = labels[:, num_soft_tokens:]
    
    predictions = np.argmax(predictions, axis=-1)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}



training_args = TrainingArguments(
    output_dir="gpt2_soft_prompt",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=15,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    report_to="none",
    fp16=True,
    eval_accumulation_steps=1,
    save_safetensors=False,
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train, 
    eval_dataset=tokenized_validation,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

trainer.train()


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,8.685900,9.335599,0.084700,0.009400,0.064500,0.081500,400.000000
2,8.396700,8.931857,0.086300,0.008700,0.065200,0.082400,400.000000
3,8.230400,8.612606,0.088100,0.008500,0.066000,0.084300,400.000000
4,8.035900,8.389034,0.088200,0.008100,0.066000,0.084400,400.000000
5,7.875800,8.204077,0.087700,0.007900,0.066300,0.084500,400.000000
6,7.784300,8.057652,0.087900,0.008000,0.066400,0.084900,400.000000
7,7.679600,7.930903,0.088100,0.007400,0.066100,0.084700,399.990000
8,7.599900,7.831755,0.088600,0.007200,0.066500,0.085100,400.000000
9,7.520200,7.745013,0.089000,0.007100,0.066600,0.085600,400.000000
10,7.447700,7.676820,0.089200,0.007600,0.066300,0.085700,400.000000


TrainOutput(global_step=36000, training_loss=7.597370930989583, metrics={'train_runtime': 30437.1879, 'train_samples_per_second': 17.741, 'train_steps_per_second': 1.183, 'total_flos': 0.0, 'train_loss': 7.597370930989583, 'epoch': 20.0})